In [66]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import json 


# load to df function

In [67]:
def file_to_df(file_path):
    file_path = file_path
    with open(file_path,'r')as f:
        data = json.load(f)
    df = pd.DataFrame(data['data'])
    return df


# time stamp preparation function

In [68]:


def prep_timestamp_column(df):
    """
    Prepare timestamp column by standardizing to milliseconds

    Parameters:
    -----------
    df : pandas.DataFrame
        Input DataFrame with timestamp column
    Returns:
    --------
    pandas.DataFrame
        DataFrame with converted timestamp column
    """
    # Identify the timestamp column
    possible_cols = ['time', 'timestamp']
    col_name = next((col for col in possible_cols if col in df.columns), None)
    if col_name is None:
        raise ValueError("DataFrame must contain either 'time' or 'timestamp' column")
    

    # Vectorized standardization
    def standardize_timestamps(series):
        # Convert to numeric, coercing errors to NaN
        numeric_series = pd.to_numeric(series, errors='coerce')
        # Handle NaN values
        numeric_series = numeric_series.fillna(0)
        # Convert to string and handle integer/float inputs
        ts_str = numeric_series.astype(str).str.split('.').str[0]
        # Trim long timestamps
        ts_str = ts_str.str.slice(0, 13)
        # Pad short timestamps
        ts_str = ts_str.str.pad(13, side='right', fillchar='0')
        return ts_str.astype(np.int64)
    
    # Standardize timestamps
    df['timestamp'] = standardize_timestamps(df[col_name])
    # Convert to datetime using milliseconds
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    # Drop the original column if it's different from 'timestamp'
    if col_name != 'timestamp':
        df.drop(columns=[col_name], inplace=True)
    
    
    return df

In [69]:
# drop time received and set timestamp first column
def drop_and_reorganise(df):
    df.drop(columns=['time_received'], inplace=True)
    df = df[['timestamp','asks','bids']]
    return df

# call all functions to load and prep timestamp

In [70]:

level2_deep5_filepath = '/root/trading_systems/kucoin_dir/kucoin_release_data_initial/2024-12-08_11-40_XRP/XRP_level2Depth5_data.json'

df_deep5 = file_to_df(level2_deep5_filepath)
df_deep5 = prep_timestamp_column(df_deep5)
df_deep5 = drop_and_reorganise(df_deep5)
df_deep5


,timestamp,asks,bids
0,2024-12-08 11:40:00.545,"[[2.54541, 156.4134], [2.54627, 2678.2151], [2...","[[2.5454, 2967.9417], [2.54537, 293.2806], [2...."
1,2024-12-08 11:40:00.640,"[[2.54595, 135.3617], [2.54614, 585], [2.54677...","[[2.54594, 585], [2.54585, 199.0214], [2.54583..."
2,2024-12-08 11:40:00.745,"[[2.54631, 585], [2.54636, 310.8617], [2.54637...","[[2.54605, 284.8], [2.54602, 230.9656], [2.545..."
3,2024-12-08 11:40:00.855,"[[2.54606, 135.3617], [2.54626, 175.5], [2.546...","[[2.54605, 869.8], [2.54602, 230.9656], [2.545..."
4,2024-12-08 11:40:00.958,"[[2.54615, 135.3617], [2.54619, 320.91], [2.54...","[[2.54614, 869.8], [2.54611, 585], [2.54605, 3..."
...,...,...,...
32595,2024-12-08 12:40:22.545,"[[2.56, 41.7522], [2.56005, 245.6918], [2.5600...","[[2.55998, 585], [2.55983, 98.486], [2.55972, ..."
32596,2024-12-08 12:40:22.732,"[[2.56, 41.7522], [2.56005, 245.6918], [2.5600...","[[2.55998, 585], [2.55983, 98.486], [2.55972, ..."
32597,2024-12-08 12:40:23.050,"[[2.56, 41.7522], [2.56007, 39.5865], [2.56008...","[[2.55998, 585], [2.55967, 282.2749], [2.55966..."
32598,2024-12-08 12:40:23.153,"[[2.56, 626.7522], [2.56007, 39.5865], [2.5600...","[[2.55981, 256.3], [2.55979, 125.2577], [2.559..."


# filter and create columns
- for best bid and ask and mean price 
- weighted_avg_ask_price and weighted_avg_bid_price
- imbalance

In [71]:


def prep_deep5_df(df):
    df = df.copy()
    # Convert lists to numpy arrays
    asks_array = np.array(df['asks'].tolist(), dtype=float)
    bids_array = np.array(df['bids'].tolist(), dtype=float)
    # Access the first value of each array
    first_ask_values = asks_array[:, 0, 0]
    first_bid_values = bids_array[:, 0, 0]

    # create column with best ask and best bid and average of the two
    df['best_ask'] = first_ask_values
    df['best_bid'] = first_bid_values
    df['mean_price'] = df[['best_ask','best_bid']].mean(axis=1)
    # df['mean_price'] = (first_ask_values + first_bid_values) / 2

    # Extract prices and sizes
    ask_prices = asks_array[:, :, 0]
    ask_sizes = asks_array[:, :, 1]
    bid_prices = bids_array[:, :, 0]
    bid_sizes = bids_array[:, :, 1]

    # Calculate weighted average prices
    weighted_avg_ask_price = np.sum(ask_prices * ask_sizes, axis=1) / np.sum(ask_sizes, axis=1)
    weighted_avg_bid_price = np.sum(bid_prices * bid_sizes, axis=1) / np.sum(bid_sizes, axis=1)

    # Add the results to the DataFrame
    df['weighted_avg_ask_price'] = weighted_avg_ask_price
    df['weighted_avg_bid_price'] = weighted_avg_bid_price

    # Calculate total bid size and total ask size
    total_bid_size = np.sum(bid_sizes, axis=1)
    total_ask_size = np.sum(ask_sizes, axis=1)

    # Calculate imbalance
    imbalance = (total_bid_size - total_ask_size) / (total_bid_size + total_ask_size)

    # Add the imbalance to the DataFrame
    df['imbalance'] = imbalance
    df.drop(['asks','bids'], axis=1, inplace=True)
    return df

df_deep5 = prep_deep5_df(df_deep5)
df_deep5.to_pickle('df_deep5_preped.pkl')
df_deep5

,timestamp,best_ask,best_bid,mean_price,weighted_avg_ask_price,weighted_avg_bid_price,imbalance
0,2024-12-08 11:40:00.545,2.54541,2.54540,2.545405,2.546273,2.545377,-0.410726
1,2024-12-08 11:40:00.640,2.54595,2.54594,2.545945,2.546560,2.545861,-0.148564
2,2024-12-08 11:40:00.745,2.54631,2.54605,2.546180,2.546385,2.545984,-0.236727
3,2024-12-08 11:40:00.855,2.54606,2.54605,2.546055,2.546317,2.545839,0.394562
4,2024-12-08 11:40:00.958,2.54615,2.54614,2.546145,2.546236,2.546113,-0.021551
...,...,...,...,...,...,...,...
32595,2024-12-08 12:40:22.545,2.56000,2.55998,2.559990,2.560195,2.559769,-0.185042
32596,2024-12-08 12:40:22.732,2.56000,2.55998,2.559990,2.560195,2.559769,-0.185042
32597,2024-12-08 12:40:23.050,2.56000,2.55998,2.559990,2.560115,2.559720,0.345011
32598,2024-12-08 12:40:23.153,2.56000,2.55981,2.559905,2.560071,2.559724,0.140983
